# 02.2 - Kedro on Databricks, part 2

_Notes: This notebook is supposed to be run locally from VS Code, all with Databricks Connect_

## Kedro and Databricks Connect

**Databricks Connect** is a client library that allows you to run Spark code locally on your machine while connecting to a remote Databricks cluster for computation. It essentially lets you develop and execute Spark applications from your local IDE or notebook environment, but the actual processing happens on the Databricks cluster.

The **Databricks extension for Visual Studio Code** has several interesting features for connecting to Databricks from VS Code and perform actions sach us deploying and running Databricks Asset Bundles, manage clusters, and easily set up **Databricks Connect**.

Therefore, the two are the perfect companion for developing Kedro projects on VS Code, since you can develop on your IDE while using Databricks compute.

Follow the official documentation to

1. [Install the Databricks extension for VS Code](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/install)
2. [Configure the appropriate cluster](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/configure)
3. [Install Databricks Connect](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/databricks-connect)

_Note: `databricks-connect` provides its own `pyspark` top-level module, and [pip doesn't check for conflicting packages](https://github.com/pypa/pip/issues/4625), so make sure you don't have a [conflicting `pyspark` installation](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-connect/python/troubleshooting#conflicting-pyspark-installations)!_

### Install the needed requirements again

In [ ]:
%pip install -r ../../../requirements.in
%pip install hdfs s3fs

Uninstall dependencies that conflict:

In [4]:
%pip uninstall -y kedro-mlflow

Found existing installation: kedro_mlflow 0.14.4
Uninstalling kedro_mlflow-0.14.4:
  Successfully uninstalled kedro_mlflow-0.14.4
Note: you may need to restart the kernel to use updated packages.


Load the Kedro ipython extension.

In [5]:
%load_ext kedro.ipython

[06/16/25 10:45:21] INFO     Using                                                                  ]8;id=350227;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=502506;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/si                
                             te-packages/kedro/framework/project/rich_logging.yml' as logging                      
                             configuration.                                                                        

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=800185;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=768958;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=31000;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=466533;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=930013;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=448021;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/Merel_Theisen/Projects/kedro-academy/kedro-databricks-bootcamp/                
                             02_databricks/rocketfuel.                                                             
                             To set a different path, run '%reload_kedro <project_root>'                           

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/io/core.p │
│ y:181 in from_config                                                                             │
│                                                                                                  │
│   178 │   │                                                                                      │
│   179 │   │   """                                                                                │
│   180 │   │   try:                                                                               │
│ ❱ 181 │   │   │   class_obj, config = parse_dataset_definition(                                  │
│   182 │   │   │   │   config, load_version, save_version                                         │
│   183 │   │   │   )                                                                              │
│   184 │   │   except Exception as exc:                                                           │
│                                                                                                  │
│ /Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/io/core.p │
│ y:517 in parse_dataset_definition                                                                │
│                                                                                                  │
│   514 │   │   │   │   "`pip install kedro-datasets[<dataset-group>]` to install `kedro-dataset   │
│   515 │   │   │   │   "related dependencies for the specific dataset group."                     │
│   516 │   │   │   )                                                                              │
│ ❱ 517 │   │   │   raise DatasetError(                                                            │
│   518 │   │   │   │   f"Class '{dataset_type}' not found, is this a typo?" f"\n{hint}"           │
│   519 │   │   │   )                                                                              │
│   520                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
DatasetError: Class 'kedro_mlflow.io.models.MlflowModelTrackingDataset' not found, is this a typo?
Hint: If you are trying to use a dataset from `kedro-datasets`, make sure that the package is installed in your 
current environment. You can do so by running `pip install kedro-datasets` or `pip install 
kedro-datasets[<dataset-group>]` to install `kedro-datasets` along with related dependencies for the specific 
dataset group.

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_ipython().run_line_magic('load_ext', 'kedro.ipython')                                    │
│   2                                                                                              │
│                                                                                                  │
│ /Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/IPython/core/in │
│ teractiveshell.py:2480 in run_line_magic                                                         │
│                                                                                                  │
│   2477 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2478 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2479 │   │   │   with self.builtin_trap:                     

Ensure your project now contains the databricks configuration created in the previous notebook. Add it into your `base/catalog.yml` file. The content should be as follows:

```yaml
_uc_catalog: #your catalog location
_uc_schema: #your schema location

companies_raw:
  type: spark.SparkDataset
  filepath: /Volumes/${_uc_catalog}/${_uc_schema}/bronze/companies.csv
  file_format: csv
  load_args:
    header: True
    inferSchema: True

reviews_raw:
  type: spark.SparkDataset
  filepath: /Volumes/${_uc_catalog}/${_uc_schema}/bronze/reviews.csv
  file_format: csv
  load_args:
    header: True
    inferSchema: True

companies:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: companies
  write_mode: overwrite

reviews:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: reviews
  write_mode: overwrite

shuttles:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: shuttles
  write_mode: overwrite

preprocessed_companies:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: preprocessed_companies
  write_mode: overwrite

preprocessed_shuttles:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: preprocessed_shuttles
  write_mode: overwrite

preprocessed_reviews:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: preprocessed_reviews
  write_mode: overwrite

model_input_table:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: model_input_table
  write_mode: overwrite

regressor:
  type: pickle.PickleDataset
  filepath: data/06_models/regressor.pickle
  versioned: true
```

You should also have a `conf/local/catalog.yml` file that contains the following:

```yaml
shuttles_raw:
  type: pandas.ExcelDataset
  filepath: data/01_raw/shuttles.xlsx
  load_args:
    engine: openpyxl
```

In [ ]:
%reload_kedro ../../rocketfuel

In [ ]:
catalog.list()

Notice how data is loaded as a PySpark DataFrame, directly from Databricks Unity Catalog!

In [ ]:
catalog._get_dataset("companies")

In [ ]:
display(catalog.load("companies"))

### Exercise 1

Codify the logic of the dummy `load_data` pipeline inside the project and run it locally through `databricks-connect`. For that:

- Create the pipeline in your local development environment.
- Run ```%reload_kedro``` to reload the Kedro project.
- Try to execute the `load_data` pipeline from the VS Code notebook.
- Iterate until it works.

In [ ]:
%reload_kedro ../../rocketfuel

In [ ]:
# The below command let's you execute the `load_data` pipeline
session.run("load_data")

## Integration with Databricks MLflow

### Log Kedro runs as MLflow experiments

There are 2 types of MLflow experiments in Databricks:
- **Workspace** experiments are not associated with any notebook, and any notebook can log a run to these experiments by using the experiment ID or the experiment name. _They cannot be created inside Git folders._
- **Notebook** experiments are associated with a specific notebook. _They are note checked into source control_.

Therefore, for personal experimentation **notebook** experiments are more appropriate, and for collaboration **workspace** experiments can be created in a regular workspace folder outside of Git.

Since you will be running this notebook locally using Databricks Connect, creating a **workspace** experiment will be more flexible. First, create the appropriate parent directory using the Databricks SDK:

In [1]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

current_user = w.current_user.me()
home_dir = f"/Users/{current_user.user_name}"
home_dir

'/Users/merel_theisen@mckinsey.com'

Next, you will need a Databricks token:

In [ ]:
# FIXME: Call `databricks configure` on the CLI instead of setting the environment variables here?
#
# import os
#
# os.environ["DATABRICKS_INSTANCE"] = w.config.host
# Do NOT commit this to version control!
# os.environ["DATABRICKS_TOKEN"] = "..."


Finally, verify that everything works:

In [2]:
import mlflow

# This workaround is needed with serverless compute, see official answer at
# https://community.databricks.com/t5/machine-learning/using-datbricks-connect-with-serverless-compute-and-mlflow/m-p/97604#M3764
mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = (
    lambda: "databricks-uc"
)

experiment_path = f"{home_dir}/02_2-kedro-on-databricks"

mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_path)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3389653495272219', creation_time=1749817173167, experiment_id='3389653495272219', last_update_time=1750063246513, lifecycle_stage='active', name='/Users/merel_theisen@mckinsey.com/02_2-kedro-on-databricks', tags={'mlflow.experiment.sourceName': '/Users/merel_theisen@mckinsey.com/02_2-kedro-on-databricks',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'merel_theisen@mckinsey.com',
 'mlflow.ownerId': '1425391920417431'}>

MLflow is the perfect companion for Kedro projects, thanks to the `kedro-mlflow` community plugin:

In [6]:
%pip install kedro-mlflow

  Using cached kedro_mlflow-0.14.4-py3-none-any.whl.metadata (14 kB)
Using cached kedro_mlflow-0.14.4-py3-none-any.whl (54 kB)
Note: you may need to restart the kernel to use updated packages.


`kedro-mlflow` can take [configuration](https://kedro-mlflow.readthedocs.io/en/0.14.4/source/03_experiment_tracking/01_experiment_tracking/01_configuration.html) from `conf/<environment>/mlflow.yml`, which can be used to configure the experiment name.

To this end, let's add some OmegaConf syntax to `mlflow.yml` so that the experiment name can be specified from the outside:

In [ ]:
%%writefile ../conf/local/mlflow.yml
server:
  mlflow_tracking_uri: databricks

tracking:
  experiment:
    name: ${runtime_params:mlflow_experiment_name}

Now you can pass the experiment name as a runtime parameter:

_Note: Extra params cannot contain spaces when passed to `%reload_kedro`, see [this issue](https://github.com/kedro-org/kedro/issues/4813)_

In [3]:
%reload_kedro --params mlflow_experiment_name=$experiment_path

UsageError: Line magic function `%reload_kedro` not found.


Now, every time a Kedro pipeline is run, it's logged as al MLflow run:

In [ ]:
session.run("data_processing")

![MLflow run corresponding to a Kedro run on Databricks](./kedro-databricks-mlflow-run.png)

### Exercise 3

Make the necessary changes to the project so that you can run a Kedro pipeline and log the results as a MLflow experiment from the CLI:

```bash
(.venv) $ kedro run -p data_processing --params mlflow_experiment_name=/Users/juan_luis_cano@mckinsey.com/02_2-kedro-on-databricks
```

_Note: To run locally you might need to `export DATABRICKS_SERVERLESS_COMPUTE_ID=auto`, see https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-connect/cluster-config#configure-a-connection-to-serverless-compute_

### Register models using the Databricks Unity Catalog

Registering models on the Unity Catalog from Kedro pipelines is trivial.

Adjust the `regressor` dataset:

```diff
 regressor:
-  type: pickle.PickleDataset
-  filepath: data/06_models/regressor.pickle
-  versioned: true
+  type: kedro_mlflow.io.models.MlflowModelTrackingDataset
+  flavor: mlflow.sklearn
```

And run the `data_science` pipeline:

In [7]:
%reload_kedro --params mlflow_experiment_name=$experiment_path

[06/16/25 10:45:32] INFO     Resolved project path as:                                              ]8;id=897938;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=245686;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/Merel_Theisen/Projects/kedro-academy/kedro-databricks-bootcamp/                
                             02_databricks/rocketfuel.                                                             
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=697211;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=388674;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

[06/16/25 10:45:33] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=602210;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=523521;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[06/16/25 10:45:34] INFO     Kedro project rocketfuel                                               ]8;id=209355;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=948864;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=72225;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=266101;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=329503;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=133182;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [ ]:
session.run("data_science")

[06/16/25 10:45:35] INFO     Kedro project rocketfuel                                                ]8;id=972545;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/framework/session/session.py\session.py]8;;\:]8;id=361411;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro/framework/session/session.py#329\329]8;;\

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=827763;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=738592;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

[06/16/25 10:45:36] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=856118;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=987079;file:///Users/Merel_Theisen/anaconda3/envs/iris-databricks/lib/python3.10/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

![Model registry](kedro-databricks-mlflow-model-registry.png)